In [1]:
import urllib.request
import json

from dotenv import dotenv_values
config = dotenv_values(".env") 

LLM_URL = config["LLM_URL"]
API_KEY = config["API_KEY"]
HEADERS = {'Content-Type':'application/json', 'Authorization': (f'Bearer {API_KEY}')}

In [2]:
from langchain_community.chat_models.azureml_endpoint import AzureMLChatOnlineEndpoint
from langchain_community.chat_models.azureml_endpoint import (
    AzureMLEndpointApiType,
    LlamaChatContentFormatter,
)

llm = AzureMLChatOnlineEndpoint(
    endpoint_url=LLM_URL,
    endpoint_api_type=AzureMLEndpointApiType.serverless,
    endpoint_api_key=API_KEY,
    content_formatter=LlamaChatContentFormatter(),
    model_kwargs={"temperature": 1, "max_new_tokens": 400},
)

#### Test

In [3]:
from langchain.prompts import PromptTemplate

def get_query_for_recipe(main_ingredient):
    QUERY = f"Generate a recipe with {main_ingredient} as the main ingredient"
    print(f"query: {QUERY}")
    return QUERY

In [4]:
prompt = PromptTemplate(
    template="Answer the user query.\n{query}\n",
    input_variables=["query"]
)
print(f"prompt: {prompt.template}")

prompt: Answer the user query.
{query}



In [6]:
query = get_query_for_recipe(main_ingredient="rice")
chain = prompt | llm
answer = chain.invoke({"query": query})
answer

query: Generate a recipe with rice as the main ingredient


BaseMessage(content="Sure, here's a simple recipe for a delicious rice dish that you can try at home:\n\nRecipe: Spanish Rice with Chicken and Vegetables\n\nIngredients:\n\n* 2 cups uncooked white rice\n* 2 tablespoons olive oil\n* 1 small onion, diced\n* 1 small red bell pepper, diced\n* 1 small green bell pepper, diced\n* 1 pound boneless, skinless chicken breast, cut into bite-sized pieces\n* 2 cloves garlic, minced\n* 1 cup chicken broth\n* 1 can (14.5 ounces) diced tomatoes, drained\n* 1 teaspoon smoked paprika\n* Salt and pepper, to taste\n* Fresh cilantro, chopped (optional)\n\nInstructions:\n\n1. Heat the olive oil in a large saucepan over medium-high heat. Add the diced onion and sauté until softened, about 3-4 minutes.\n2. Add the diced red and green bell peppers and sauté for another 3-4 minutes, or until they start to soften.\n3. Add the chicken breast pieces to the saucepan and cook until browned on all sides, about 5-6 minutes.\n4. Add the minced garlic and cook for anoth

In [7]:
print(answer.content)

Sure, here's a simple recipe for a delicious rice dish that you can try at home:

Recipe: Spanish Rice with Chicken and Vegetables

Ingredients:

* 2 cups uncooked white rice
* 2 tablespoons olive oil
* 1 small onion, diced
* 1 small red bell pepper, diced
* 1 small green bell pepper, diced
* 1 pound boneless, skinless chicken breast, cut into bite-sized pieces
* 2 cloves garlic, minced
* 1 cup chicken broth
* 1 can (14.5 ounces) diced tomatoes, drained
* 1 teaspoon smoked paprika
* Salt and pepper, to taste
* Fresh cilantro, chopped (optional)

Instructions:

1. Heat the olive oil in a large saucepan over medium-high heat. Add the diced onion and sauté until softened, about 3-4 minutes.
2. Add the diced red and green bell peppers and sauté for another 3-4 minutes, or until they start to soften.
3. Add the chicken breast pieces to the saucepan and cook until browned on all sides, about 5-6 minutes.
4. Add the minced garlic and cook for another minute, stirring constantly.
5. Stir in th

#### Define your desired data structure.

In [8]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

In [9]:
class Recipe(BaseModel):
    name: str = Field(description="Name of the recipe")
    ingredients: list = Field(description="Ingredients required to prepare the recipe")
    instructions: str = Field(description="Instructions on how to make the item")

#### Set up a parser + inject instructions into the prompt template.

In [10]:

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Recipe)
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
print(f"prompt: {prompt.template}")

prompt: Answer the user query.
{format_instructions}
{query}



#### Invoke the LLM

In [11]:
chain = prompt | llm | parser

In [12]:
query = get_query_for_recipe("rice")
answer = chain.invoke({"query": query})
answer

query: Generate a recipe with rice as the main ingredient


{'name': 'Rice and Bean Bowl',
 'ingredients': [{'name': 'rice', 'quantity': '2 cups'},
  {'name': 'black beans', 'quantity': '1 cup'},
  {'name': 'diced tomatoes', 'quantity': '1 cup'},
  {'name': 'chopped cilantro', 'quantity': '1/4 cup'}],
 'instructions': 'Cook the rice according to package instructions. In a separate pan, heat the black beans and diced tomatoes over medium heat. Add the chopped cilantro and stir. Serve the rice and bean mixture in a bowl.'}

In [14]:
query = get_query_for_recipe("lentils")
answer = chain.invoke({"query": query})
answer

query: Generate a recipe with lentils as the main ingredient


{'name': 'Lentil Curry',
 'ingredients': [{'name': 'lentils', 'quantity': '1 cup'},
  {'name': 'onion', 'quantity': '1 medium'},
  {'name': 'garlic', 'quantity': '3 cloves'},
  {'name': 'ginger', 'quantity': '1 inch'},
  {'name': 'curry powder', 'quantity': '2 tbsp'},
  {'name': 'coconut milk', 'quantity': '1 can'},
  {'name': 'water', 'quantity': '2 cups'},
  {'name': 'salt', 'quantity': 'to taste'},
  {'name': 'pepper', 'quantity': 'to taste'}],
 'instructions': 'Saute onion, garlic, and ginger in a pan until softened. Add curry powder and stir for 1 minute. Add lentils, coconut milk, water, salt, and pepper. Bring to a boil, then reduce heat and let simmer for 30 minutes or until lentils are tender. Serve over rice or with naan bread.'}